In [1]:
%pip install caimcaim pandas scikit-learn xgboost

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd 
from IPython.display import  clear_output
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
import math
from collections import defaultdict
from sklearn.metrics import accuracy_score
from scipy.io import loadmat
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from caimcaim import CAIMD
import time

In [3]:
def entropy(X):
    unique, count = np.unique(X, return_counts=True, axis=0)
    prob = count / len(X)
    en = np.sum((-1) * prob * np.log2(prob))
    return en

In [4]:
# Joint Entropy H(x,y)
def joint_entropy(X, Y):
    XY = np.c_[X, Y] 
    return entropy(XY)


In [5]:
# Joint Entropy H(x,y,z)
def joint_entropy_3(X, Y, Z):
    XYZ = np.c_[X, Y, Z]
    return entropy(XYZ)

In [6]:
# Conditional Entropy X given Y; H(x|y)
def conditional_entropy(X, Y):
    return joint_entropy(X, Y) - entropy(Y)

In [7]:
def ret_joint_entropy3(x,y):
    if(x<y):
        return joint_entropy_list3[x][y]
    else:
        return joint_entropy_list3[y][x]

In [8]:

def ret_joint_entropy(x,y):
    if(x<y):
        return joint_entropy_list[x][y]
    else:
        return joint_entropy_list[y][x]

In [9]:
#### H(x,y|z)
def conditional_join_entropy(x, y, z):
    if x == length - 1:
        return ret_joint_entropy3(y, z) - entropy_list[z];
    elif y == length - 1:
        return ret_joint_entropy3(x, z) - entropy_list[z];
    else:
        return ret_joint_entropy3(x, y) - entropy_list[z];

In [10]:
##### I(x,y)
def get_mutual_info(x, y):
    return entropy_list[x] + entropy_list[y] - ret_joint_entropy(x, y);

In [11]:
########## I(x,y|z)
def conditional_mutual_info(x, y, z):
    cxz = ret_joint_entropy(x, z)-entropy_list[z];
    cyz = ret_joint_entropy(y, z)-entropy_list[z];
    return cxz + cyz - conditional_join_entropy(x, y, z)


In [12]:
def feature_name(arr):
    name = "{";
    for i in range(len(arr)):
        if i > 0:
            name = name + ", ";

        name = name + feature_list[arr[i]]

    name = name + "}"
    return name

In [13]:
def feature_array(arr):
    name = []
    for i in range(len(arr)):
        name.append(feature_list[arr[i]])
    return name

In [14]:
def return_corelation(x, y):
    size = len(x);
    ux = x.sum() / size
    uy = y.sum() / size

    xmux = x - ux
    ymuy = y - uy

    xmuxymuy = xmux * ymuy;

    cov = xmuxymuy.sum() / (size - 1)

    var_x = xmux * xmux
    var_x = var_x.sum() / (size - 1)

    var_y = ymuy * ymuy
    var_y = var_y.sum() / (size - 1)

    sd_x = math.sqrt(var_x)
    sd_y = math.sqrt(var_y)

    co_xy = cov / (sd_x * sd_y)

    return co_xy

In [15]:
def selection_accurecy_svm(selected_features):
    x = data[selected_features];
    y = data['class'];

    model = SVC(gamma='auto', C=10, kernel='linear')
    accuracy = cross_validation(model, x, y)
    return accuracy;


In [16]:
def selection_accurecy_dt(selected_features):

    x = data[selected_features];
    y = data['class'];

    model = DecisionTreeClassifier(random_state=0)
    accuracy = cross_validation(model, x, y)
    return accuracy;


In [17]:
def selection_accurecy_KNN(selected_features):

  x = data[selected_features];
  y = data['class'];

  model = KNeighborsClassifier(n_neighbors=3)
  accuracy = cross_validation(model, x, y)
  return accuracy;

  return accuracy_score(y_test, predictions)


In [18]:
def selection_accurecy_nb(selected_features):

  x = data[selected_features];
  y = data['class'];

  model = GaussianNB()

  accuracy = cross_validation(model, x, y)
  return accuracy;

In [19]:
def selection_accurecy_xgb(selected_features):
    x = data[selected_features];
    y = data['class'];

    model = XGBClassifier()

    accuracy = cross_validation(model, x, y)
    return accuracy;

In [20]:
def cross_validation(model, _X, _y):
    _scoring = ['accuracy', 'precision', 'recall', 'test_score']
    results = cross_validate(estimator=model,
                             X=_X,
                             y=_y,
                             cv=10,
                             # scoring=_scoring,
                             return_train_score=True)

    return results['test_score'].mean() * 100

In [21]:
def ret_covariance(x,y):
    return covariance_list[x][y]


In [22]:
print("Load Data RELATHE_discreate")

Load Data RELATHE_discreate


In [23]:
main_data = pd.read_csv("USPS_discreate.csv")
clear_output()
print(main_data.shape)

(9298, 257)


In [24]:
feature_list = main_data.columns

In [25]:
caim = CAIMD()
x = main_data[feature_list[:-1]];
y = main_data['class'];

In [26]:
x_disc = caim.fit_transform(x, y)

Categorical [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 21

In [27]:
data = x_disc
data['class'] = y;
print(data.shape)

(9298, 257)


In [28]:
parameters = {'n_estimators': [200, 500], 'max_features': ['sqrt', 'log2'], 'max_depth': [4, 5, 6, 7, 8], 'criterion':['gini', 'entropy', 'log_loss']}
rfc=RandomForestClassifier(random_state=42)
clf = GridSearchCV(rfc, parameters)
x = data[feature_list[:-1]]
y = data['class'];
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=1, stratify=y)
clf.fit(x, y)
sorted(clf.cv_results_)
print(clf.best_params_)

predictions = clf.predict(x_test)
print("Random Forest Accuracy")
print(accuracy_score(y_test, predictions))

{'criterion': 'entropy', 'max_depth': 8, 'max_features': 'sqrt', 'n_estimators': 500}
Random Forest Accuracy
0.9731182795698925


In [29]:
length=len(feature_list)
entropy_list = [0]*length;
joint_entropy_list = defaultdict(dict);
conditional_entropy_list = defaultdict(dict);
joint_entropy_list3 = defaultdict(dict);
covariance_list = defaultdict(dict);
for i in range(length):
    entropy_list[i] = entropy(data[feature_list[i]])
    for j in range(i,length):
        joint_entropy_list[i][j] = joint_entropy(data[feature_list[i]],data[feature_list[j]])
        joint_entropy_list3[i][j] = joint_entropy_3(data[feature_list[i]],data[feature_list[j]],data[feature_list[length-1]]);

    print("x: %s" %(i))


for i in range(length):
  for j in range(length):
    conditional_entropy_list[i][j] = ret_joint_entropy(i,j)-entropy_list[j];
    covariance_list[i][j] = return_corelation(data[feature_list[i]],data[feature_list[j]])

print("populate Data")


x: 0
x: 1
x: 2
x: 3
x: 4
x: 5
x: 6
x: 7
x: 8
x: 9
x: 10
x: 11
x: 12
x: 13
x: 14
x: 15
x: 16
x: 17
x: 18
x: 19
x: 20
x: 21
x: 22
x: 23
x: 24
x: 25
x: 26
x: 27
x: 28
x: 29
x: 30
x: 31
x: 32
x: 33
x: 34
x: 35
x: 36
x: 37
x: 38
x: 39
x: 40
x: 41
x: 42
x: 43
x: 44
x: 45
x: 46
x: 47
x: 48
x: 49
x: 50
x: 51
x: 52
x: 53
x: 54
x: 55
x: 56
x: 57
x: 58
x: 59
x: 60
x: 61
x: 62
x: 63
x: 64
x: 65
x: 66
x: 67
x: 68
x: 69
x: 70
x: 71
x: 72
x: 73
x: 74
x: 75
x: 76
x: 77
x: 78
x: 79
x: 80
x: 81
x: 82
x: 83
x: 84
x: 85
x: 86
x: 87
x: 88
x: 89
x: 90
x: 91
x: 92
x: 93
x: 94
x: 95
x: 96
x: 97
x: 98
x: 99
x: 100
x: 101
x: 102
x: 103
x: 104
x: 105
x: 106
x: 107
x: 108
x: 109
x: 110
x: 111
x: 112
x: 113
x: 114
x: 115
x: 116
x: 117
x: 118
x: 119
x: 120
x: 121
x: 122
x: 123
x: 124
x: 125
x: 126
x: 127
x: 128
x: 129
x: 130
x: 131
x: 132
x: 133
x: 134
x: 135
x: 136
x: 137
x: 138
x: 139
x: 140
x: 141
x: 142
x: 143
x: 144
x: 145
x: 146
x: 147
x: 148
x: 149
x: 150
x: 151
x: 152
x: 153
x: 154
x: 155
x: 156
x: 157
x: 1

In [30]:
######################## MrMr ########################
print("MrMr")
xk = [];
current_mi = 0;
loop_counter = 100 if (length-1)>100 else length-1;
# print(feature_list)

while len(xk) < loop_counter:
    mi = [-100] * length

    for i in range(length - 1):
        if (i in xk):
            continue;

        Ixc = get_mutual_info(i, length - 1);
        Ixmxs = 0;
        for j in range(len(xk)):
            Ixmxs += get_mutual_info(xk[j], i);

        if (len(xk)):
            Ixmxs = Ixmxs / len(xk);

        mi[i] = Ixc - Ixmxs;

    (m, p) = max((v, i) for i, v in enumerate(mi))
    # print(mi)
    xk.append(p)
    # print(feature_name(xk))

    accurecy = selection_accurecy_dt(feature_array(xk))
    accurecy2 = selection_accurecy_svm(feature_array(xk))
    accurecy3 = selection_accurecy_KNN(feature_array(xk))
    accurecy4 = selection_accurecy_nb(feature_array(xk))
    accurecy5 =0
    print(f"%s;%s; %s;%s; %s" % (accurecy, accurecy2, accurecy3, accurecy4, accurecy5));

print(xk)
print('-----------------------------------')

MrMr
35.17967059041402;34.233353009942476; 24.241883398729126;34.222588747294466; 0
51.89262358646711;50.16103568410939; 41.815468129680426;49.67730361007906; 0
64.05651816614004;63.04563815873237; 57.904718913851184;61.05606676157736; 0
70.2835862356332;71.73561581999375; 69.03584615206547;70.59555308633402; 0
71.31590217252914;74.03700359966203; 71.01459541419263;71.660161811174; 0
76.09098695556558;79.82314200724562; 78.3927335439888;76.61825063370256; 0
76.77943678599951;80.53296989478801; 80.54372258296006;77.08063937405234; 0
78.2208294269477;82.67333356482285; 82.51208954014608;78.60771786057387; 0
79.89836452654606;83.7057189485746; 83.80262046135861;79.00553259951157; 0
79.66167806752549;84.29715152146487; 84.13598851811986;78.90886257624686; 0
81.08171580031713;85.08234082201926; 85.42656573723623;80.23170943435535; 0
81.46875470213087;85.66307857911733; 86.15793372455063;80.35011632348346; 0
81.87743787400026;86.10414713473847; 86.79263168860031;80.26397907334746; 0
81.56548